In [2]:
from csv import DictReader

In [10]:
reader = DictReader(open('podatki/st_in_vrednost_nepremicnin.csv', 'rt', encoding='utf-8'))

#len(reader.fieldnames)



for col in reader.fieldnames:
    print(col)
    for row in reader:
        print(row[col])
    reader = DictReader(open('podatki/st_in_vrednost_nepremicnin.csv', 'rt', encoding='utf-8'))


DWELLINGS
1 Dwellings - TOTAL
1.1 Newly built dwellings
1.1.1 Newly built flats
1.1.2 Newly built family houses
1.2 Existing dwellings
1.2.1 Existing flats, Slovenia
1.2.2 Existing family houses
2010Q1 Number of transactions
1961
216
161
55
1745
1336
409
2010Q1 Value of housing transactions (EUR)
192589161
26886114
18652307
8233807
165703047
117833352
47869695
2010Q2 Number of transactions
2068
236
158
78
1832
1354
478
2010Q2 Value of housing transactions (EUR)
208839168
33032927
19946719
13086208
175806241
121325887
54480354
2010Q3 Number of transactions
1813
213
162
51
1600
1171
429
2010Q3 Value of housing transactions (EUR)
182716681
26398295
18324172
8074123
156318386
104910420
51407966
2010Q4 Number of transactions
1939
196
157
39
1743
1280
463
2010Q4 Value of housing transactions (EUR)
194310186
26633062
19886384
6746678
167677124
112863610
54813514
2011Q1 Number of transactions
1874
174
137
37
1700
1311
389
2011Q1 Value of housing transactions (EUR)
181801783
23471327
16942227
6